In [1]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                   SCF-TB - PROXY APPLICATION                      %
# %                   A.M.N. Niklasson, M. Kulichenko. T1, LANL       %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# % Total Energy Function:                                            %
# % E = 2Tr[H0(D-D0)] + (1/2)*sum_i U_i q_i^2 +                       %
# %      + (1/2)sum_{i,j (i!=j)} q_i C_{ij} q_j - Efield*dipole       %
# % dipole = sum_i R_{i} q_i                                          %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

import torch
import warnings
import logging
import os
# to disable torchdynamo completely. Faster for smaller systems and single-point calculations.
os.environ["TORCHDYNAMO_DISABLE"] = "1"  # hard-disable capture

import numpy as np
import matplotlib.pyplot as plt

from dftorch.SCF import SCFx, SCFx_batch
from dftorch.Constants import Constants
from dftorch.Structure import Structure, StructureBatch
from dftorch.MD import MDXL, MDXLBatch
from dftorch.ESDriver import ESDriver, ESDriverBatch

### Configure torch and torch.compile ###
# Silence warnings and module logs
warnings.filterwarnings("ignore")
os.environ["TORCH_LOGS"] = ""               # disable PT2 logging
os.environ["TORCHINDUCTOR_VERBOSE"] = "0"
os.environ["TORCHDYNAMO_VERBOSE"] = "0"
logging.getLogger("torch.fx").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.symbolic_shapes").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.recording").setLevel(logging.CRITICAL)
# Enable dynamic shape capture for dynamo
torch._dynamo.config.capture_dynamic_output_shape_ops = True
# default data type
torch.set_default_dtype(torch.float64)

torch.cuda.empty_cache()

# MD with PBC and PME
### Note, PME does not work properly for small cells (< 20x20x20)
### Also, do not use PME without PBC.

## First, SCF and forces

In [2]:
%%time
dftorch_params = {
    'coul_method': 'PME', # 'FULL' for full coulomb matrix, 'PME' for PME method
    'Coulomb_acc': 5e-5,   # Coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 12.0,        # Coulomb cutoff
    'PME_order': 4,        # Ignored for FULL coulomb method

    'SCF_MAX_ITER': 100,    # Maximum number of SCF iterations
    'SCF_TOL': 1e-6,       # SCF convergence tolerance on density matrix
    'SCF_ALPHA': 0.05,      # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 30,  # Maximum Krylov subspace rank
    'KRYLOV_TOL': 1e-6,    # Krylov subspace convergence tolerance in SCF
    'KRYLOV_TOL_MD': 1e-4, # Krylov subspace convergence tolerance in MD SCF
    'KRYLOV_START': 50,     # Number of initial SCF iterations before starting Krylov acceleration
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#filename = '8RNT.xyz'      
filename = 'COORD.xyz'            # Solvated acetylacetone and glycine molecules in H20, Na, Cl
LBox = torch.tensor([45.0, 45.0, 40.0], device=device) # Simulation box size in Angstroms. Only cubic boxes supported for now.
# Create constants container. Set path to SKF files.
const = Constants(filename,
                  #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
                  'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\ptbp\\complete_set\\',
                  magnetic_hubbard_ldep=False
                  ).to(device)

# Create structure object. Define total charge and electronic temperature.
structure1 = Structure(filename, LBox, const, charge=7, Te=500.0, device=device)

# Create ESDriver object and run SCF calculation
# electronic_rcut and repulsive_rcut are in Angstroms.
# They should be >= cutoffs defined in SKF files for the element pair with largest cutoff present in the system.
es_driver = ESDriver(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const) # Calculate forces after SCF

int64
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\H-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\H-C.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\C-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\ptbp\complete_set\C-C.skf
### Do SCF ###
  Initial DM_Fermi

Starting cycle
Iter 1
Res = 4.970438028, dEc = 168.497638350, t = 0.3 s

Iter 2
Res = 4.507938368, dEc = 4.576993912, t = 0.1 s

Iter 3
Res = 4.079268757, dEc = 3.873256898, t = 0.1 s

Iter 4
Res = 3.675967503, dEc = 3.285308046, t = 0.1 s

Iter 5
Res = 3.292975209, dEc = 2.790016098, t = 0.1 s

Iter 6
Res = 2.927810751, dEc = 2.372232943, t = 0.0 s

Iter 7
Res = 2.581068374, dEc = 2.020433760, t = 0.0 s

Iter 8
Res = 2.256647291, dEc = 1.724898423, t = 0.0 s

Iter 9
Res = 1.960484610, dEc = 1.477199476, t = 0.0 s

Iter 10
Res = 1.698078518, dEc = 1.270210122, t = 0.0 s

Iter 11
Res = 1.472613435, dEc = 1.097811204, t = 0.0 s

Iter 12
Res = 1.284742613, dEc = 0.9

In [3]:
#charges:
structure1.q

tensor([-0.2438, -0.7706, -0.4516, -0.3950, -0.7243,  0.2517, -0.4235, -0.4312,
        -0.3921, -0.4288,  0.2427, -0.3826, -0.4541, -0.4357, -0.4114, -0.2073,
        -0.3938, -0.7813, -0.7546, -0.4127])

In [4]:
#MO coeffs
structure1.Q

tensor([[ 1.7793e-01, -3.5631e-01,  4.9032e-01,  ..., -2.9076e-01,
          3.5272e-01, -2.0136e-01],
        [-6.8824e-02,  8.9261e-02, -1.3560e-02,  ..., -1.9987e-01,
          8.2232e-02,  1.5840e-02],
        [-2.2904e-04,  1.4454e-03, -2.4638e-04,  ..., -8.8955e-02,
         -2.0499e-02,  5.5811e-02],
        ...,
        [-3.3998e-02, -8.0351e-02, -1.3895e-01,  ..., -1.3726e-01,
         -1.1252e-01, -6.3736e-02],
        [-3.9725e-02, -9.2969e-02, -1.5810e-01,  ..., -2.6449e-01,
         -2.3171e-01, -1.4924e-01],
        [-1.4647e-01, -2.3625e-01, -2.1485e-01,  ..., -2.5871e-01,
         -3.5329e-01, -3.0009e-01]])

In [5]:
print("Total Energy: ", structure1.e_tot)
print("Chemical Potential: ", structure1.mu0)

Total Energy:  tensor(-138.3300)
Chemical Potential:  tensor(-44.0957)


## MD

In [6]:
torch.manual_seed(0)
temperature_K = torch.tensor(1.0, device=structure1.device)
mdDriver = MDXL(es_driver, const, temperature_K=temperature_K)
# Set number of steps, time step (fs), dump interval and trajectory filename
mdDriver.run(structure1, dftorch_params, num_steps=100, dt=0.3, dump_interval=5, traj_filename='md_trj.xyz')

AssertionError: Torch not compiled with CUDA enabled

# Batched MD with PBC and full Coulomb calculations.
### Systems in a batch should have same number of atoms and orbitals. Number of electrons and atom order can be arbitrary.
### Note, Batched mode does not support PME, only full Coulomb.

In [2]:
%%time
dftorch_params = {
    'coul_method': 'FULL',
    'Coulomb_acc': 1e-5, # coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 10.0, # coulomb cutoff
    'PME_order': 4,

    'SCF_MAX_ITER': 20,
    'SCF_TOL': 1e-6,
    'SCF_ALPHA': 0.3, # Linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 30,
    'KRYLOV_TOL': 1e-6,
    'KRYLOV_TOL_MD': 1e-4,
    'KRYLOV_START': 3,
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda'

filename = ['COORD_8WATER.xyz']*4
LBox = torch.tensor([[21,21,21], [22,22,22], [21,21,22], [22,22,21]], device=device)

const = Constants(
    filename,
    '/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set').to(device)

structure = StructureBatch(
    filename, LBox, const, charge=0, Te=3000.0, device=device)

es_driver = ESDriverBatch(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure, const, do_scf=True, verbose=False)
es_driver.calc_forces(structure, const)

/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-O.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/O-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/O-O.skf
   LMAX: tensor([7, 8, 7, 8], device='cuda:0', dtype=torch.int32)

Starting cycle
Iter 1
Batch 0: Res = 1.474e+00, dEc = 2.273e+00
Batch 1: Res = 1.474e+00, dEc = 2.272e+00
Batch 2: Res = 1.474e+00, dEc = 2.272e+00
Batch 3: Res = 1.474e+00, dEc = 2.272e+00
Iter 2
Batch 0: Res = 7.799e-01, dEc = 1.464e+00
Batch 1: Res = 7.800e-01, dEc = 1.464e+00
Batch 2: Res = 7.800e-01, dEc = 1.464e+00
Batch 3: Res = 7.800e-01, dEc = 1.464e+00
Iter 3
Batch 0: Res = 4.153e-01, dEc = 7.168e-01
Batch 1: Res = 4.154e-01, dEc = 7.169e-01
Batch 2: Res = 4.153e-01, dEc = 7.169e-01
Batch 3: Res = 4.153e-01, dEc = 7.168e-01
Iter 4
  rank: 0, batch 0, Fel = 0.002370
  rank: 0, batch 1, Fel = 0.002369
  rank: 0, batch 2, 

In [3]:
torch.manual_seed(0)
temperature_K = torch.tensor([20.0, 30.0, 30.0, 30.0], device=structure.device)
mdDriver = MDXLBatch(es_driver, const, temperature_K=temperature_K)
mdDriver.run(structure, dftorch_params, num_steps=60, dt=0.3, dump_interval=5, traj_filename='batch_md_trj.xyz')

########## Step = 0 ##########
   LMAX: tensor([7, 8, 7, 8], device='cuda:0', dtype=torch.int32)
H0: 1.003 s
H1: 0.208 s
  rank: 0, batch 0, Fel = 0.000168
  rank: 0, batch 1, Fel = 0.000177
  rank: 0, batch 2, Fel = 0.000123
  rank: 0, batch 3, Fel = 0.000133
  Not converged: 4
  rank: 1, batch 0, Fel = 0.000015
  rank: 1, batch 1, Fel = 0.000019
  rank: 1, batch 2, Fel = 0.000010
  rank: 1, batch 3, Fel = 0.000010
  Not converged: 0
KER: 0.186 s
F AND E: 0.474 s
ETOT = -873.74561658, EPOT = -873.94177829, EKIN = 0.05687440, T = 18.33331009, ResErr = 0.000000, t = 1.9 s
ETOT = -873.71672304, EPOT = -873.93116957, EKIN = 0.08531159, T = 27.49996514, ResErr = 0.000000, t = 1.9 s
ETOT = -873.71691834, EPOT = -873.89832728, EKIN = 0.08531159, T = 27.49996514, ResErr = 0.000000, t = 1.9 s
ETOT = -873.71698197, EPOT = -873.90553529, EKIN = 0.08531159, T = 27.49996514, ResErr = 0.000000, t = 1.9 s
0.01258496 GB


########## Step = 1 ##########
   LMAX: tensor([7, 8, 7, 8], device='cuda:0', d

KeyboardInterrupt: 